# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [2]:
#1 Set the file path to import the WeatherPy_database.csv file
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saskylakh,RU,71.9167,114.0833,3.31,99,67,5.77,broken clouds
1,1,Atuona,PF,-9.8000,-139.0333,77.18,74,31,17.34,scattered clouds
2,2,Udachnyy,RU,66.4167,112.4000,30.45,92,100,11.23,snow
3,3,Hermanus,ZA,-34.4187,19.2345,65.80,85,81,14.76,broken clouds
4,4,Lubango,AO,-14.9172,13.4925,63.77,78,42,2.39,scattered clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [3]:
#2a Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip?"))

#2b Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the max temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?5
What is the max temperature you would like for your trip?545


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [4]:
#3 Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df =city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                      (city_data_df["Max Temp"] >= min_temp)]
                                    
# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Atuona,PF,-9.8000,-139.0333,77.18,74,31,17.34,scattered clouds
2,2,Udachnyy,RU,66.4167,112.4000,30.45,92,100,11.23,snow
3,3,Hermanus,ZA,-34.4187,19.2345,65.80,85,81,14.76,broken clouds
4,4,Lubango,AO,-14.9172,13.4925,63.77,78,42,2.39,scattered clouds
5,5,Khatanga,RU,71.9667,102.5000,6.96,88,33,8.81,scattered clouds
6,6,Mataura,NZ,-46.1927,168.8643,44.29,98,100,6.40,moderate rain
7,7,Nikolskoye,RU,59.7035,30.7861,42.69,88,100,5.64,overcast clouds
8,8,Pangody,RU,65.8500,74.4859,27.68,90,100,11.77,overcast clouds
9,9,Bluff,NZ,-46.6000,168.3333,46.38,80,100,18.81,overcast clouds
10,10,Bredasdorp,ZA,-34.5322,20.0403,63.09,84,83,10.56,broken clouds


In [5]:
#4 determine if there are any empty rows
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
#4b drop them if necessary 
clean_df = preferred_cities_df.dropna().copy()
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Atuona,PF,-9.8000,-139.0333,77.18,74,31,17.34,scattered clouds
2,2,Udachnyy,RU,66.4167,112.4000,30.45,92,100,11.23,snow
3,3,Hermanus,ZA,-34.4187,19.2345,65.80,85,81,14.76,broken clouds
4,4,Lubango,AO,-14.9172,13.4925,63.77,78,42,2.39,scattered clouds
5,5,Khatanga,RU,71.9667,102.5000,6.96,88,33,8.81,scattered clouds
6,6,Mataura,NZ,-46.1927,168.8643,44.29,98,100,6.40,moderate rain
7,7,Nikolskoye,RU,59.7035,30.7861,42.69,88,100,5.64,overcast clouds
8,8,Pangody,RU,65.8500,74.4859,27.68,90,100,11.77,overcast clouds
9,9,Bluff,NZ,-46.6000,168.3333,46.38,80,100,18.81,overcast clouds
10,10,Bredasdorp,ZA,-34.5322,20.0403,63.09,84,83,10.56,broken clouds


In [7]:
clean_df.count()

City_ID                653
City                   653
Country                653
Lat                    653
Lng                    653
Max Temp               653
Humidity               653
Cloudiness             653
Wind Speed             653
Current Description    653
dtype: int64

In [8]:
#5a Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

In [9]:
#5b Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Atuona,PF,77.18,scattered clouds,-9.8000,-139.0333,
2,Udachnyy,RU,30.45,snow,66.4167,112.4000,
3,Hermanus,ZA,65.80,broken clouds,-34.4187,19.2345,
4,Lubango,AO,63.77,scattered clouds,-14.9172,13.4925,
5,Khatanga,RU,6.96,scattered clouds,71.9667,102.5000,
6,Mataura,NZ,44.29,moderate rain,-46.1927,168.8643,
7,Nikolskoye,RU,42.69,overcast clouds,59.7035,30.7861,
8,Pangody,RU,27.68,overcast clouds,65.8500,74.4859,
9,Bluff,NZ,46.38,overcast clouds,-46.6000,168.3333,
10,Bredasdorp,ZA,63.09,broken clouds,-34.5322,20.0403,


In [10]:
# 6a Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [11]:
# 7 Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[(hotel_df['Hotel Name'].str.len() > 0)]
clean_hotel_df.count()

City                   106
Country                106
Max Temp               106
Current Description    106
Lat                    106
Lng                    106
Hotel Name             106
dtype: int64

In [12]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
40,Asau,RO,47.32,scattered clouds,46.4333,26.4000,Casa Mika Ciobanus
64,Ancud,CL,48.81,light rain,-41.8697,-73.8203,Hotel Arena Gruesa
68,Hobart,AU,51.66,broken clouds,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania
100,Soyo,AO,77.20,broken clouds,-6.1349,12.3689,Hospedaria ndozi'ame
111,Grand-Santi,GF,84.61,overcast clouds,4.2500,-54.3833,Hotel La Villa Couacou
115,Port Hardy,CA,46.58,light rain,50.6996,-127.4199,Airport Inn-Port Hardy
121,Kindu,CD,71.65,broken clouds,-2.9500,25.9500,Résidence Okanda
126,Del Rio,US,80.62,few clouds,29.3627,-100.8968,"Holiday Inn Express & Suites del Rio, an IHG H..."
148,Longyearbyen,SJ,19.24,few clouds,78.2186,15.6401,"Radisson Blu Polar Hotel, Spitsbergen"
166,Goderich,CA,73.72,scattered clouds,43.7501,-81.7165,Harmony Inn


In [13]:
# 8a Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b Display the figure
fig

Figure(layout=FigureLayout(height='420px'))